# BYTE-PAIR-ENCODING del Quijote
--- 
Este es una forma simple de compresión de datos en la que el par más común de bytes consecutivos de datos se reemplaza con un byte que no ocurre dentro de esos datos. Aquí, el objetivo no es la compresión de datos, sino la codificación de texto en un idioma dado como una secuencia de 'tokens', utilizando un vocabulario fijo de diferentes tokens. La mayoría de las palabras se codificarán como un solo token, mientras que las palabras raras se codificarán como una secuencia de unos pocos tokens, donde estos tokens representan partes de palabras significativas.

### Equipo cangrejo      
* Montaño Preciado Alondra Karolina
* Velasquez Hidalgo Luis Juventino
* Navarro Lopez Malcom Hiram
* Faz Leal Juan Carlos


### Fuentes
- Medium con la informacion: https://towardsdatascience.com/byte-pair-encoding-subword-based-tokenization-algorithm-77828a70bee0

- Codigo en el cual nos estamos inspirando: https://leimao.github.io/blog/Byte-Pair-Encoding/

In [ ]:
import re, collections

### get_vocab()
___
La función get_vocab lee un archivo de texto (especificado por filename) y devuelve un diccionario de frecuencias de palabras (vocabulario).
Cada línea en el archivo de texto se divide en palabras y cada palabra se agrega al vocabulario con un contador de frecuencia inicial de 1.
Si la palabra ya existe en el vocabulario, su contador se incrementa en 1.
- filename: Es un argumento de entrada para la función, es el nombre (o la ruta) del archivo de texto que se va a leer para crear el vocabulario.
- return: Es una instrucción que se utiliza para devolver un valor desde la funcion, en este caso, la función get_vocab devuelve el diccionario de frecuencias

de palabras (vocabulario) creado en el cuerpo de la función, que se almacena en la variable vocab.


In [ ]:
def get_vocab(filename):
    '''
    Parte el texto si encuentra espacios
    '''
    vocab = collections.defaultdict(int)
    with open(filename, 'r', encoding='utf-8') as fhand:
        for line in fhand:
            words = line.strip().split()
            for word in words:
                vocab[' '.join(list(word)) + ' </w>'] += 1
    return vocab

### get_stats()
___
Aqui se explica la funcion de esta celda


In [ ]:
def get_stats(vocab):
    '''
    Devuelve el numero de veces que se repiten las palabras
    '''
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

### merge_vocab()
___
Funcion que junta los dos tokens que mas aparecen consecutivamente en el vocabulatio.
El bigrama es la union de los dos tokens que mas aparecen. 

Si la palabra contiene el bigrama, "w_out" obtiene el valor de la palabra actual remplazando los dos tokens por el bigrama, de lo contrario "w_out" obtiene conserva el valor de la palabra de 

la instruccion:
`v_out[w_out] = v_in[word]`
nos asegura que la frecuencia de la *posible* nueva palabra se conserva.
- pari: El par de tokens que mas se repiten  
- v_in: El vocabulario actual de palabras.
- return: El nuevo vocabulario, donde la ocurrencia del par de tokens 'pair' se remplazo por su union.

In [ ]:
def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

### get_tokens_from_vocab()
___
Aqui se explica la funcion de esta celda

In [ ]:
def get_tokens_from_vocab(vocab):
    tokens_frequencies = collections.defaultdict(int)
    vocab_tokenization = {}
    for word, freq in vocab.items():
        word_tokens = word.split()
        for token in word_tokens:
            tokens_frequencies[token] += freq
        vocab_tokenization[''.join(word_tokens)] = word_tokens
    return tokens_frequencies, vocab_tokenization

### measure_token_length()
___

In [ ]:
def measure_token_length(token):
    if token[-4:] == '</w>':
        return len(token[:-4]) + 1
    else:
        return len(token)

Aqui se explica la funcion de esta celda

In [ ]:
def tokenize_word(string, sorted_tokens, unknown_token='</u>'):
    
    if string == '':
        return []
    if sorted_tokens == []:
        return [unknown_token]

    string_tokens = []
    for i in range(len(sorted_tokens)):
        token = sorted_tokens[i]
        token_reg = re.escape(token.replace('.', '[.]'))

        matched_positions = [(m.start(0), m.end(0)) for m in re.finditer(token_reg, string)]
        if len(matched_positions) == 0:
            continue
        substring_end_positions = [matched_position[0] for matched_position in matched_positions]

        substring_start_position = 0
        for substring_end_position in substring_end_positions:
            substring = string[substring_start_position:substring_end_position]
            string_tokens += tokenize_word(string=substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
            string_tokens += [token]
            substring_start_position = substring_end_position + len(token)
        remaining_substring = string[substring_start_position:]
        string_tokens += tokenize_word(string=remaining_substring, sorted_tokens=sorted_tokens[i+1:], unknown_token=unknown_token)
        break
    return string_tokens

## Implementacion usando el Quijote
---


Aqui implementamos el codigo de Byte-pair-encoding

In [ ]:
vocab = get_vocab('TextoEjemplo.txt')

num_merges = 10000
for i in range(num_merges):
    pairs = get_stats(vocab)
    print("Pares: ", pairs)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    print('All tokens: {}'.format(tokens_frequencies.keys()))
    print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
    print('==========')

In [ ]:
'''
print('==========')
print('Tokens Before BPE')
tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
print('All tokens: {}'.format(tokens_frequencies.keys()))
print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
print('==========')

num_merges = 2
for i in range(num_merges):
    pairs = get_stats(vocab)
    if not pairs:
        break
    best = max(pairs, key=pairs.get)
    vocab = merge_vocab(best, vocab)
    print('Iter: {}'.format(i))
    print('Best pair: {}'.format(best))
    tokens_frequencies, vocab_tokenization = get_tokens_from_vocab(vocab)
    print('All tokens: {}'.format(tokens_frequencies.keys()))
    print('Number of tokens: {}'.format(len(tokens_frequencies.keys())))
    print('==========')

# Let's check how tokenization will be for a known word
word_given_known = 'mountains</w>'
word_given_unknown = 'Ilikeeatingapples!</w>'

sorted_tokens_tuple = sorted(tokens_frequencies.items(), key=lambda item: (measure_token_length(item[0]), item[1]), reverse=True)
sorted_tokens = [token for (token, freq) in sorted_tokens_tuple]

print(sorted_tokens)

word_given = word_given_known 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))

word_given = word_given_unknown 

print('Tokenizing word: {}...'.format(word_given))
if word_given in vocab_tokenization:
    print('Tokenization of the known word:')
    print(vocab_tokenization[word_given])
    print('Tokenization treating the known word as unknown:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
else:
    print('Tokenizating of the unknown word:')
    print(tokenize_word(string=word_given, sorted_tokens=sorted_tokens, unknown_token='</u>'))
'''

# Conclusiones
___
Me diverti mucho